In [1]:
# https://github.com/mrthlinh/Spotify-Playlist-Recommender
# https://towardsdatascience.com/building-music-playlists-recommendation-system-564a3e63ef64
# https://github.com/rn5l/session-rec/tree/master/algorithms/knn
# https://onezero.medium.com/how-spotifys-algorithm-knows-exactly-what-you-want-to-listen-to-4b6991462c5c

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.sparse import csr_matrix

In [2]:
playlists = pd.read_csv('Copy of explicit_data - Playlist_data.csv')
user_playlist_data = pd.read_csv('Copy of explicit_data - User_playlists.csv')
songs = pd.read_csv('Copy of explicit_data - Songs - All.csv')

In [3]:
print(playlists.head())
# print(user_playlist_data.head())
# print(songs.head())

   playlist_id               playlist_name  user_id
0          101            Top Sinhala Hits    10001
1          102           New Sinhala songs    10004
2          103      Sinhala Rap Collection    10019
3          104  Christmas Songs Collection    10022
4          105              Mother's Songs    10065


In [17]:
print(user_playlist_data.head())

   playlist_id     timestamp  order  song_id
0          101  1.396767e+09     25      312
1          101  1.396767e+09     26      313
2          101  1.396767e+09     24      314
3          101  1.396767e+09     23      315
4          101  1.396767e+09     22      316


In [7]:
print(songs.head())

  song_id                          Title  \
0       2                    Aa Ra Sulan   
1     283                     Aale katha   
2       3      Ada Nam Ma Hada Iwasum Na   
3       4  Ada Thaniyen Ma Hadanne Na Ma   
4       5         Adambarai Baluwama Nam   

                                         Artist  Artist_id        Album  \
0                              Nirosha Virajini       21.0  Aa Ra Sulan   
1  Kalpana Nayanamadu, Shermaine Willis ft Iraj       11.0   Aale Katha   
2             Raveen Kanishka & Kalpana Kavindi      101.0          NaN   
3                              Shihan Mihiranga       62.0          NaN   
4                                 Surani De Mel       80.0          NaN   

   Release Year  
0        2011.0  
1        2018.0  
2           NaN  
3           NaN  
4           NaN  


In [36]:
user_playlist_data.dtypes

playlist_id      int64
timestamp      float64
order            int64
song_id          int64
dtype: object

In [38]:
songs.dtypes

song_id           int64
Title            object
Artist           object
Artist_id       float64
Album            object
Release Year    float64
dtype: object

In [4]:
df_merge = pd.merge(user_playlist_data, songs.drop_duplicates(['song_id']), on="song_id", how="left")

In [5]:
df_merge.head()

,playlist_id,timestamp,order,song_id,Title,Artist,Artist_id,Album,Release Year
0,101,1.396767e+09,25,312,Walakulaka Kandulu Piruna,Uresha Ravihari,138.0,NaN,NaN
1,101,1.396767e+09,26,313,Umathu Prema Kumara (Chanchala Hagumaka Gilila),Shashika Nisansala,84.0,NaN,NaN
2,101,1.396767e+09,24,314,Tharu Kada Watunu Dura Akase,Saman Lenin,139.0,NaN,NaN
3,101,1.396767e+09,23,315,Siththamai Numba,Dayan Witharana & Samadhi Arunachaya,140.0,NaN,NaN
4,101,1.396767e+09,22,316,Seetha Ra Weluna Horen,Ranil Fernando M.A.,141.0,NaN,NaN


In [41]:
df_merge.shape

(59, 9)

In [6]:
TopSinhalaHits = df_merge.loc[df_merge['playlist_id']==101]
TopSinhalaHits.head()

,playlist_id,timestamp,order,song_id,Title,Artist,Artist_id,Album,Release Year
0,101,1.396767e+09,25,312,Walakulaka Kandulu Piruna,Uresha Ravihari,138.0,NaN,NaN
1,101,1.396767e+09,26,313,Umathu Prema Kumara (Chanchala Hagumaka Gilila),Shashika Nisansala,84.0,NaN,NaN
2,101,1.396767e+09,24,314,Tharu Kada Watunu Dura Akase,Saman Lenin,139.0,NaN,NaN
3,101,1.396767e+09,23,315,Siththamai Numba,Dayan Witharana & Samadhi Arunachaya,140.0,NaN,NaN
4,101,1.396767e+09,22,316,Seetha Ra Weluna Horen,Ranil Fernando M.A.,141.0,NaN,NaN


In [7]:
TopSinhalaHits.shape

(26, 9)

In [9]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import FastText
from sklearn.neighbors import NearestNeighbors

In [10]:
def buildFTModel(playlist_id):
    
    # extract unique words
    playlist = df_merge.loc[df_merge['playlist_id']==playlist_id]
    song_names = playlist.Title.values
    song_name_clean = [re.sub(r'[^\w]', ' ', str(item))for item in song_names]
    song_name_clean = [re.sub(r" \d+", '', str(item.strip())) for item in song_name_clean]
    
    sentences = list()
    for item in song_name_clean:
        sentences.append(item.split())
#     unique_sentence = sentences # build the model on all unique sentence but not all sentence to save time
    
    # BUild FastText Model
    num_features = 50    # Word vector dimensionality                      
    min_word_count = 1                      
    num_workers = 1      # Number of CPUs
    context = 3          # Context window size; 

    downsampling = 1e-3   # threshold for configuring which 
                          # higher-frequency words are randomly downsampled

    # Initialize and train the model 
    model = FastText(workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling, sg = 1)

    model.build_vocab(sentences = sentences)
    model.train(sentences = sentences,  total_examples=len(sentences), epochs=10)

    model.init_sims(replace=True)
    
    model.save('playlist.model')
    model = FastText.load('playlist.model')
    return model

In [11]:
model = buildFTModel(101)

In [ ]:
def generate_similar_items(dataset):
    song_titles = dataset.Title.values
    for 

In [14]:
song_titles = songs.Title.values
song_titles
# for i in song_titles:
#     print(i)

# for i in song_titles:
#     positives = i.split()
model.similarity('Ae')

Aa Ra Sulan
Aale katha
Ada Nam Ma Hada Iwasum Na
Ada Thaniyen Ma Hadanne Na Ma
Adambarai Baluwama Nam
Adanne Ay Sudu Manike
Adaraneeya Neranjana 
Adaraya Ayai
Adare sithum
Adarema Geethayak
Adaren (Lanwenna Hithuwata)
Aduru kutiya thula 
Ae
Ahasin eha
Ahasin polowata
Ai Kale Adare
Ai kale mulu hadinma
Akeekaru pem kathawak
Alawanthakam
Alen Ma
Alen Wela Ganna
Amma Sandaki
Anagathaye
Ananthayata Yana Para Dige
Ananthaye 
Anatha maruthe 
Api hagum walata ida dee mohothak
Api kauruda
Arabumama Kadulak Wela Ma Bala Iddi
Atha Kadukara Himau Arane
Atha Ran Wiman Thulin Pata Selayen Sadi
Athinwath atha
Athithaya Sihinayak Pamanai
Athsana
Awado Sansare Ma Ha Badee
Awathan hade
Ay Kale Mulu Hadinma Oba Mata Adare
Ayage Sinaha
Baila Gamuda
Billa
 Sina Podak Wee
Chain
Chakithaya
Chandani Payala
Chandrayan Pidu Kiranak Sagawala Horen
Channa Kinnaravi
Ciao Malli
Cuore Nero (Italian)
Daasin Paa
Daffodil mala
Daiwaye Saradamin
Das Piyan Wesena Asille
Dasama riddana
Dasin Pa Ma Ingi Maru Dewliye
Dedun

In [3]:
# let's create a song playlist matrix
def song_palylist_matrix(user_playlist_data, songs):
    sp_ratings = pd.merge(user_playlist_data, songs.drop_duplicates(['song_id']), on="song_id", how="left")
    sp_ratings['rates'] = np.ones((59,), dtype=int)
    
    # pivot ratings into song features
    sp_ratings_matrix = sp_ratings.pivot(
        index='playlist_id',
        columns='song_id',
        values='rates'
    ).fillna(0)
    
    return sp_ratings_matrix


In [43]:
print(song_palylist_matrix(user_playlist_data, songs))

song_id      7    10   13   14   19   22   23   24   25   26   ...  315  316  \
playlist_id                                                    ...             
101          1.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  ...  1.0  1.0   
102          0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
103          0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   
104          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
105          0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
109          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

song_id      317  318  319  320  321  322  323  324  
playlist_id                                          
101          1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  
102          0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
103          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
104          0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  
105          0.0  0.0  0.0  0.0  0.

In [48]:
def sparse_matrix(user_playlist_data, songs):
    sp_matrix = song_palylist_matrix(user_playlist_data, songs)
    user_playlist_mat = csr_matrix(sp_matrix.values)
    return user_playlist_mat

In [51]:
print(sparse_matrix(user_playlist_data, songs))

  (0, 0)	1.0
  (0, 1)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 13)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 21)	1.0
  (0, 37)	1.0
  (0, 40)	1.0
  (0, 41)	1.0
  (0, 42)	1.0
  (0, 45)	1.0
  (0, 46)	1.0
  (0, 47)	1.0
  (0, 48)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 51)	1.0
  (0, 52)	1.0
  :	:
  (1, 29)	1.0
  (1, 30)	1.0
  (1, 31)	1.0
  (1, 32)	1.0
  (1, 37)	1.0
  (1, 38)	1.0
  (1, 43)	1.0
  (1, 44)	1.0
  (1, 54)	1.0
  (2, 6)	1.0
  (2, 10)	1.0
  (2, 24)	1.0
  (2, 25)	1.0
  (2, 28)	1.0
  (2, 33)	1.0
  (2, 34)	1.0
  (2, 35)	1.0
  (2, 39)	1.0
  (3, 55)	1.0
  (3, 56)	1.0
  (3, 57)	1.0
  (4, 2)	1.0
  (4, 36)	1.0
  (5, 22)	1.0
  (5, 23)	1.0


///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [53]:
# let's take the features of songs inside the playlist using content based approach and 
# then check the similarity with the songs in the database and give recomemndations.

In [55]:
# content based model using TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76


In [5]:
# let's take playlists with seed tracks
user_playlist_data['playlist_id'].value_counts()

101    26
102    17
103     9
104     3
109     2
105     2
Name: playlist_id, dtype: int64

In [9]:
for i in playlists['playlist_id']:
    print(playlists['playlist_name'].loc[user_playlist_data['playlist_id'] == i])

0              Top Sinhala Hits
1             New Sinhala songs
2        Sinhala Rap Collection
3    Christmas Songs Collection
4                Mother's Songs
5           Sinhala Cover Songs
6       Sinhala Buddhists Songs
7        My Old Hits Collection
8         Iraj Songs Collection
9          BnS Songs Collection
Name: playlist_name, dtype: object
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)
Series([], Name: playlist_name, dtype: object)


In [15]:
sp_ratings = pd.merge(user_playlist_data, songs.drop_duplicates(['song_id']), on="song_id", how="left")
sp_ratings.head()

,playlist_id,timestamp,order,song_id,Title,Artist,Artist_id,Album,Release Year
0,101,1.396767e+09,25,312,Walakulaka Kandulu Piruna,Uresha Ravihari,138.0,NaN,NaN
1,101,1.396767e+09,26,313,Umathu Prema Kumara (Chanchala Hagumaka Gilila),Shashika Nisansala,84.0,NaN,NaN
2,101,1.396767e+09,24,314,Tharu Kada Watunu Dura Akase,Saman Lenin,139.0,NaN,NaN
3,101,1.396767e+09,23,315,Siththamai Numba,Dayan Witharana & Samadhi Arunachaya,140.0,NaN,NaN
4,101,1.396767e+09,22,316,Seetha Ra Weluna Horen,Ranil Fernando M.A.,141.0,NaN,NaN


In [16]:
TopSinhalaHits = sp_ratings.loc[sp_ratings['playlist_id']==101]
TopSinhalaHits.head()

,playlist_id,timestamp,order,song_id,Title,Artist,Artist_id,Album,Release Year
0,101,1.396767e+09,25,312,Walakulaka Kandulu Piruna,Uresha Ravihari,138.0,NaN,NaN
1,101,1.396767e+09,26,313,Umathu Prema Kumara (Chanchala Hagumaka Gilila),Shashika Nisansala,84.0,NaN,NaN
2,101,1.396767e+09,24,314,Tharu Kada Watunu Dura Akase,Saman Lenin,139.0,NaN,NaN
3,101,1.396767e+09,23,315,Siththamai Numba,Dayan Witharana & Samadhi Arunachaya,140.0,NaN,NaN
4,101,1.396767e+09,22,316,Seetha Ra Weluna Horen,Ranil Fernando M.A.,141.0,NaN,NaN


In [17]:
vectorizer = TfidfVectorizer(analyzer='word')

In [18]:
# build matrix for the topsinhalahits playlist
tfidf_matrix = vectorizer.fit_transform(TopSinhalaHits['Title'])

In [19]:
tfidf_feature_name = vectorizer.get_feature_names()

In [20]:
tfidf_matrix.shape

(26, 80)

In [23]:
tfidf_feature_name

['aa',
 'adaraya',
 'adare',
 'ai',
 'akase',
 'ananthayata',
 'api',
 'awilla',
 'ay',
 'ayai',
 'bala',
 'balum',
 'chakithaya',
 'chanchala',
 'chandani',
 'chandrayan',
 'daffodil',
 'denuwan',
 'dige',
 'dura',
 'gamen',
 'gilila',
 'gum',
 'hadinma',
 'hagumaka',
 'hanguman',
 'hanthana',
 'hanthanata',
 'hawasaka',
 'hiriwetunado',
 'hitha',
 'horen',
 'idunu',
 'kada',
 'kale',
 'kandulu',
 'kauruda',
 'ketha',
 'kiranak',
 'kirilli',
 'kolamba',
 'kumara',
 'lassanata',
 'liyumak',
 'ma',
 'mala',
 'mata',
 'mulu',
 'nade',
 'numba',
 'oba',
 'pamawee',
 'pana',
 'para',
 'payala',
 'payana',
 'pidu',
 'piruna',
 'piya',
 'prema',
 'puran',
 'ra',
 'rangana',
 'rangum',
 'sagawala',
 'sameepe',
 'sanda',
 'seetha',
 'sihine',
 'sitha',
 'siththamai',
 'tharu',
 'umathu',
 'walakulaka',
 'walapemi',
 'wasanthe',
 'watunu',
 'weluna',
 'wu',
 'yana']

In [21]:
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [24]:
cosine_similarity

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

In [22]:
indices = pd.Series(TopSinhalaHits['Title'].index)

In [ ]:
#Function to get the most similar songs
def recommend(index, method):
    id = indices[index]
    # Get the pairwise similarity scores of all books compared that book,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10['title'].iloc[books_index]

In [ ]:
# https://gist.github.com/cmben/509869669d122b8049c0
# https://towardsdatascience.com/introduction-to-two-approaches-of-content-based-recommendation-system-fc797460c18c
# https://github.com/youonf/recommendation_system/blob/master/content_based_filtering/content_based_recommender_approach1.ipynb
# https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76
# https://github.com/dzvlfi/Recommendation-System-Algorithms/blob/master/Code/hybrid_algorithm.py
# https://github.com/phoebewong/spotify-teamNPK/blob/master/src/Playlist-withmeta-subset100.ipynb
# https://towardsdatascience.com/discovering-similarities-across-my-spotify-music-using-data-clustering-and-visualization-52b58e6f547b
# https://github.com/taylorhawks/RNN-music-recommender/tree/master/data-wrangling
# https://github.com/wsmiles000/CS109a-Spotify-Recommendation/blob/master/models/NNeighClassifier.py
# https://github.com/queirozfcom/recommender-systems/blob/master/code/python/src/item-based-knn/item-based-knn.py


In [2]:
playlists = pd.read_csv('Copy of explicit_data - Playlist_data.csv')
user_playlist_data = pd.read_csv('Copy of explicit_data - User_playlists.csv')
songs = pd.read_csv('Copy of explicit_data - Songs - All.csv')

In [4]:
# let's create a song playlist matrix
def song_palylist_matrix(user_playlist_data, songs):
    sp_ratings = pd.merge(user_playlist_data, songs.drop_duplicates(['song_id']), on="song_id", how="left")
    sp_ratings['rates'] = np.ones((59,), dtype=int)
    
    # pivot ratings into song features
    sp_ratings_matrix = sp_ratings.pivot(
        index='playlist_id',
        columns='song_id',
        values='rates'
    ).fillna(0)
    
    return sp_ratings_matrix

In [5]:
song_palylist_matrix(user_playlist_data, songs)

song_id,7,10,13,14,19,22,23,24,25,26,...,315,316,317,318,319,320,321,322,323,324
playlist_id,,,,,,,,,,,,,,,,,,,,,
101,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
103,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
105,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
